In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical


### Load and preprocess data

In [ ]:
def create_mock_data(label, size=1000):
    X = np.random.rand(size, 5)
    y = np.full((size,), label)
    return X, y

X_ddos, y_ddos = create_mock_data(0)
X_malware, y_malware = create_mock_data(1)
X_failure, y_failure = create_mock_data(2)

X_combined = np.vstack([X_ddos, X_malware, X_failure])
y_combined = np.hstack([y_ddos, y_malware, y_failure])
y_combined_cat = to_categorical(y_combined, num_classes=3)
X_combined = X_combined.reshape(-1, 5, 1)

X_train, X_test, y_train, y_test = train_test_split(X_combined, y_combined_cat, test_size=0.2, random_state=42)


### Model Architecture

In [ ]:
input_layer = Input(shape=(5, 1))
x = Conv1D(64, kernel_size=3, activation='relu')(input_layer)
x = LSTM(32)(x)
x = Dense(16, activation='relu')(x)
x = Dropout(0.2)(x)
output_layer = Dense(3, activation='softmax')(x)

model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer=Adam(clipnorm=1.0), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


### Model Training

In [ ]:
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5, batch_size=32)


### Evaluation

In [ ]:
loss, acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {acc*100:.2f}%")


### Save the Model

In [ ]:
model.save("IT_HACKATHON.h5")
